#### Sutton and Barto, Reinforcement Learning 2nd. Edition, page 92.
![Sutton and Barto, Reinforcement Learning 2nd. Edition.](./FirstVisitMCPrediction.png)

First-visit MC prediction, for estimating V

In [1]:
from env import create_standard_grid
import numpy as np
np.random.seed(42)

This function plays the game. It return the states that were visited and the rewards received.

In [2]:
def play_game(gw, policy, state=(0,0)):
    # default game starting point is state = (0,0) 
    # list of tuples that are (state, reward) pairs
 #   states_and_rewards = [(state,0)] # list of tuples that are (state, reward) pairs
    states_and_rewards = [] # list of tuples that are (state, reward) pairs
    converged = False
    while not converged:
        # get action from policy
        action = policy[state] # get action from policy
        # find reward for the action
        reward = gw.get_reward_for_action(state, action)
        # more to the new state
        stateprime = move(state,action)
        # add new state and reward to the list
        states_and_rewards.append((state,reward))
        # if you have moved to a terminal state, then stop
        if gw.is_terminal(stateprime):
            converged = True
        # update state to new state
        state = stateprime
    return states_and_rewards

def move(state, action): # only valid actions at states are sent to move
    i,j = state
    if action == 'left':
        j = j-1
    if action == 'right':
        j = j+1
    if action == 'down':
        i = i-1
    if action == 'up':
        i = i+1
    return (i,j)

Create the standard grid

In [3]:
gw = create_standard_grid()

Input policy to be evaluated

In [4]:
policy = { 
    (0,0):'up', (0,1):'right',(0,2):'right',(0,3):'up',
    (1,0):'up', (1,1):'', (1,2):'right', (1,3):'',
    (2,0):'right', (2,1):'right', (2,2):'right', (2,3):''
    }

Initialize some thing for the calculation

- Specify a discount factor - gamma
- Need a list of all states that we will use for computing returns

In [5]:
gamma = 0.9
all_states = [
            (0,0), (0,1) ,(0,2), (0,3),
            (1,0), (1,1), (1,2), (1,3),
            (2,0), (2,1), (2,2), (2,3)
]

### 1

Play a single episode of the game 

In [6]:
states_and_rewards = play_game(gw, policy)

Examine states_and_rewards list. Note states and rewards in the list.

In [7]:
states_and_rewards

[((0, 0), 0.0), ((1, 0), 0.0), ((2, 0), 0.0), ((2, 1), 0.0), ((2, 2), 1.0)]

Compute G values from states_and_rewards - **(Note there are two lists - states_and_rewards and states_and_returns)**

In [8]:
G = 0
states_and_returns = []
for s, r in reversed(states_and_rewards):
    G = r + gamma*G
    states_and_returns.append((s,G))
states_and_returns.reverse()

Look at the states_and_returns list of lists

In [9]:
states_and_returns

[((0, 0), 0.6561000000000001),
 ((1, 0), 0.7290000000000001),
 ((2, 0), 0.81),
 ((2, 1), 0.9),
 ((2, 2), 1.0)]

The seen_sets variable keeps a list of states insuring that ony the first G value is used in the calculation

In [16]:
returns = {}
for s in all_states:
    returns[s] = []
    
seen_states = set()
for s, G in states_and_returns:
    if s not in seen_states:
        returns[s].append(G)
        gw.set_value(s, np.mean(returns[s]))
        seen_states.add(s)      

In [17]:
gw.print_values()

-------------------------------------
|   0.81 |   0.90 |   1.00 |   0.00 |
-------------------------------------
|   0.73 |   0.00 |  -1.00 |   0.00 |
-------------------------------------
|   0.66 |   0.00 |  -0.90 |  -1.00 |
-------------------------------------


## 2

Repeatedly play the game ten times

In [18]:
num_episodes = 10
for t in range(num_episodes):
    states_and_rewards = play_game(gw, policy)
    G = 0
    states_and_returns = []
    for s, r in reversed(states_and_rewards):
        G = r + gamma*G
        states_and_returns.append((s,G))
    states_and_returns.reverse()
    
    returns = {}
    for s in all_states:
        returns[s] = []
    
    seen_states = set()
    for s, G in states_and_returns:
        if s not in seen_states:
            returns[s].append(G)
            gw.set_value(s, np.mean(returns[s]))
            seen_states.add(s)  

In [19]:
gw.print_values()

-------------------------------------
|   0.81 |   0.90 |   1.00 |   0.00 |
-------------------------------------
|   0.73 |   0.00 |  -1.00 |   0.00 |
-------------------------------------
|   0.66 |   0.00 |  -0.90 |  -1.00 |
-------------------------------------


Since the grid and policy are deterministic that same thing happens 10 times. 

## 3

To explore all potential paths pick a random starting point in the grid and play the game from that point to a terminal state.

In [14]:
num_episodes = 10

for t in range(num_episodes):
    
    # select a random index in the list of all states
    random_indx = np.random.randint(0,len(all_states)) 
    # get the state 
    state = all_states[random_indx]
    # if the selected random state is a barrier state or a terminal state, then do it again
    while(gw.is_barrier(state) or gw.is_terminal(state)):
        random_indx = np.random.randint(0,len(all_states))
        state = all_states[random_indx]  
        
    # play the game from the selected state
    states_and_rewards = play_game(gw, policy, state=state)
    
    # compute states_and_returns
    G = 0
    states_and_returns = []
    for s, r in reversed(states_and_rewards):
        G = r + gamma*G
        states_and_returns.append((s,G))
    states_and_returns.reverse()
    
    # initialize the returns dictionary
    returns = {}
    for s in all_states:
        returns[s] = []
    
    # perform rest of calculation on the 
    seen_states = set()
    for s, G in states_and_returns:
        if s not in seen_states:
            returns[s].append(G)
            gw.set_value(s, np.mean(returns[s]))
            seen_states.add(s)  

In [15]:
gw.print_values()

-------------------------------------
|   0.81 |   0.90 |   1.00 |   0.00 |
-------------------------------------
|   0.73 |   0.00 |  -1.00 |   0.00 |
-------------------------------------
|   0.66 |   0.00 |  -0.90 |  -1.00 |
-------------------------------------
